In [ ]:
spark.conf.set("fs.azure.account.key.stupramonitoreomercado.dfs.core.windows.net", "xxx")
spark.conf.set("fs.azure.createRemoteFileSystemDuringInitialization", "true")
dbutils.fs.ls("abfss://raw@stupramonitoreomercado.dfs.core.windows.net/")
spark.conf.set("fs.azure.createRemoteFileSystemDuringInitialization", "false")
#spark.conf.set("spark.sql.shuffle.partitions",3000)
#spark.conf.set("spark.default.parallelism",3000)

In [ ]:
dbutils.fs.ls("abfss://raw@stupramonitoreomercado.dfs.core.windows.net/OTROS/SNR/JSON/")

In [ ]:
dbutils.fs.head("abfss://raw@stupramonitoreomercado.dfs.core.windows.net/OTROS/SNR/JSON/RANGOS_AVALUOS.json")

In [ ]:
from pyspark.sql.functions import *
from pyspark.sql.types import *
'''
# https://spark.apache.org/docs/latest/sql-ref-datatypes.html

'{"SASJSONExport":"1.0",
"SASTableData+RNAGOS_AVALUOS":
[{"COD_DPTO":"05",   "COD_DANE":"05002",   "DEPARTAMENTO":"ANTIOQUIA",   ...
'''
schema = StructType().add(
  StructField("SASTableData+RNAGOS_AVALUOS", ArrayType(
      StructType([
          StructField("COD_DPTO", StringType()),   StructField("COD_DANE", StringType()), StructField("DEPARTAMENTO", StringType()), StructField("MUNICIPIO", StringType()),
          StructField("ORIP", StringType()),   StructField("VEREDA", StringType()),
          StructField("MATRICULA", StringType()),
          StructField("TIPO_PREDIO", StringType()),
          StructField("NRO_ANOTACION", StringType()),
          StructField("COD_NATURALEZA", StringType()),      StructField("NATURALEZA_JURIDICA", StringType()),   StructField("VALOR", StringType()),
          StructField("DIRECCION", StringType()),
          StructField("Divipola_Mun_0001", StringType()),
          StructField("ZRE_NOMBRE", StringType()),   StructField("SISTEMA", StringType()),  StructField("NRO_PROPIETARIOS", StringType()),
          StructField("ESTADO_FOLIO", StringType()),   StructField("AÑO RADICACION", StringType()),  StructField("LINDERO", StringType()),
          StructField("SMMLV", StringType()),   StructField("RANGOS_AVALUOS", StringType())
      ])
   )
))


df1 = spark.read.json("abfss://raw@stupramonitoreomercado.dfs.core.windows.net/OTROS/SNR/JSON/RANGOS_AVALUOS.json", schema=schema)

In [ ]:
df1.count()

In [ ]:
df1.printSchema()

In [ ]:
#df1.createOrReplaceTempView("personas")

#spark.sql("select SASTableData+RANGOS_CEDULA from personas " )

In [ ]:
#rows = df1.select(  explode( "SASTableData+RANGOS_CEDULA" )) 
#rows = df1.select( explode(  array("SASTableData+RANGOS_CEDULA").alias("row")  ) ) 

rows = df1.select( explode("SASTableData+RNAGOS_AVALUOS").alias("row") )
rows.count()

In [ ]:
rows.printSchema()

In [ ]:
display(rows)

In [ ]:
rows.select( "row.*").show()

In [ ]:
parsed_rows = rows.select( "row.*")

In [ ]:
parsed_rows_renamed = parsed_rows.withColumnRenamed("AÑO RADICACION", "ANIO_RADICACION")
#rows2.printSchema()
#rows_renamed = parsed_rows.select(col('`AÑO RADICACION`').alias("ANIO_RADICACION"))
#rows_renamed = rows.select(col("AÑO RADICACION").alias("ANIO_RADICACION")).show()
parsed_rows_renamed.printSchema()

#rows.select(rows.columns[18]).show()

In [ ]:
parsed_rows_renamed.write.parquet("abfss://standarized@stupramonitoreomercado.dfs.core.windows.net/OTROS/SNR/transacciones.parquet")

In [ ]:
df2 = spark.read.parquet("abfss://standarized@stupramonitoreomercado.dfs.core.windows.net/OTROS/SNR/transacciones.parquet")
#display(from_json(df2.select("col"))
display(df2)

In [ ]:
import numpy as np
import pandas as pd
import pandas_profiling
from pandas_profiling import ProfileReport
pd.__version__



In [ ]:
df = df2.select("*").toPandas()
profile = ProfileReport(df, title="Pandas Profiling Report")
displayHTML(profile.to_html())


In [ ]:
dbutils.fs.put("abfss://standarized@stupramonitoreomercado.dfs.core.windows.net/OTROS/SNR/transacciones.parquet_profile.html", profile.to_html() )

In [ ]:
'''
Requirements
Apache Spark>= 2.4.0
Python>=3.6

No funciona: parece que hay  incompatiblidad con versiones recientes de pandas
'''

from pyspark.sql import SparkSession
from optimus import Optimus

spark = SparkSession.builder.appName('optimus').getOrCreate()
op= Optimus(spark)

odf = op.load.parquet("abfss://standarized@stupramonitoreomercado.dfs.core.windows.net/OTROS/SNR/RANGOS_CEDULA.parquet")

In [ ]:
df_sin_duplicados = df.drop_duplicates()

In [ ]:
df_sin_duplicados.shape

In [ ]:
profile = ProfileReport(df_sin_duplicados, title="Pandas Profiling Report")
dbutils.fs.put("abfss://raw@stupramonitoreomercado.dfs.core.windows.net/OTROS/SNR/JSON/RANGOS_CEDULA_v3.parquet_profile_sin_duplicados.html", profile.to_html() )